Here we are continuing our process by taking reference from EDA notebook. As we saw there are a lot of NULL Values in price column so, we are going to drop all of the that have Null values.

In [1]:
import numpy as np 
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

C:\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_pickle('dataset/extracted_data_features')

In [3]:
data = data.loc[~data['formatted_price'].isnull()]
print(f'Products: {data.shape[0]}\n\
Features: {data.shape[1]}')

Products: 28395
Features: 7


In [4]:
data.head()

,asin,product_type_name,color,brand,title,medium_image_url,formatted_price
4,B004GSI2OS,SHIRT,Onyx Black/ Stone,FeatherLite,Featherlite Ladies' Long Sleeve Stain Resistan...,https://images-na.ssl-images-amazon.com/images...,$26.26
6,B012YX2ZPI,SHIRT,White,HX-Kingdom Fashion T-shirts,Women's Unique 100% Cotton T - Special Olympic...,https://images-na.ssl-images-amazon.com/images...,$9.99
11,B001LOUGE4,SHIRT,Black,Fitness Etc.,Ladies Cotton Tank 2x1 Ribbed Tank Top,https://images-na.ssl-images-amazon.com/images...,$11.99
15,B003BSRPB0,SHIRT,White,FeatherLite,FeatherLite Ladies' Moisture Free Mesh Sport S...,https://images-na.ssl-images-amazon.com/images...,$20.54
21,B014ICEDNA,SHIRT,Purple,FNC7C,Supernatural Chibis Sam Dean And Castiel Short...,https://images-na.ssl-images-amazon.com/images...,$7.50


In [5]:
data = data.loc[~data['color'].isnull()]
print(f'Products: {data.shape[0]}\n\
Features: {data.shape[1]}')

Products: 28385
Features: 7


In [6]:
#As we have taken down our dataset from 180k products to 28k products.
#Let's save it then
data.to_pickle('dataset/28K_apparel')

In [7]:
#Number of duplicated titles that are present in our data. Definitely we don't want to recommend the same product.
print(sum(data.duplicated('title')))

2325


In [8]:
#data[data.duplicated('title')==False]

In [9]:
#Remove all the title that are very short in the length
title_sorted = data[data['title'].apply(lambda x: len(x.split()) > 4)]
print(f'After Title sorted: {title_sorted.shape[0]}')

After Title sorted: 27949


In [10]:
title_sorted.sort_values('title', inplace=True, ascending=False, ignore_index=False)
title_sorted.head()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,asin,product_type_name,color,brand,title,medium_image_url,formatted_price
61973,B06Y1KZ2WB,SHIRT,Black/Pink,Éclair,Éclair Women's Printed Thin Strap Blouse Black...,https://images-na.ssl-images-amazon.com/images...,$24.99
133820,B010RV33VE,SHIRT,Pink,xiaoming,xiaoming Womens Sleeveless Loose Long T-shirts...,https://images-na.ssl-images-amazon.com/images...,$18.19
81461,B01DDSDLNS,SHIRT,White,xiaoming,xiaoming Women's White Long Sleeve Single Brea...,https://images-na.ssl-images-amazon.com/images...,$21.58
75995,B00X5LYO9Y,SHIRT,Red Anchors,xiaoming,xiaoming Stripes Tank Patch/Bear Sleeve Anchor...,https://images-na.ssl-images-amazon.com/images...,$15.91
151570,B00WPJG35K,SHIRT,White,xiaoming,xiaoming Sleeve Sheer Loose Tassel Kimono Woma...,https://images-na.ssl-images-amazon.com/images...,$14.32


In [11]:
#title_sorted = title_sorted.reset_index()
#title_sorted.head()

In [12]:
#title_sorted.head()

In [13]:
#title_sorted.tail()

In [14]:
indices = []
for i,row in title_sorted.iterrows():
    indices.append(i)

In [15]:
import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = title_sorted.shape[0]
while i < num_data_points and j < num_data_points:
    
    previous_i = i

    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    a = data['title'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:

        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']
        b = data['title'].loc[indices[j]].split()

        # store the maximum length of two strings
        length = max(len(a), len(b))

        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            stage1_dedupe_asins.append(title_sorted['asin'].loc[indices[i]])


            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

In [16]:
data = data.loc[data['asin'].isin(stage1_dedupe_asins)]

In [17]:
print(f'Number of products after title Cleaning: {data.shape}')

Number of products after title Cleaning: (17592, 7)


In [18]:
data.to_pickle('dataset/17k_apparel')